Things that needs to be encoded:

A full description of a chess position, i.e. the position "state", must contain the following elements:

1. The location of each piece on the board
2. Whose turn it is to move
3. Status of the 50-move draw rule. The name of this is sometimes a bit confusing, as it is 50 moves by each player, and therefore 100 half-moves, or ply. For example, if the previous 80 half-moves passed without a capture or a pawn move, the fifty-move rule will kick in after another twenty half-moves.
4. Whether either player is permanently disqualified to castle, both kingside and queenside.
5. If an en passant capture is possible.

References:

    [1] https://en.wikipedia.org/wiki/Board_representation_(computer_chess)

In [1]:
%load_ext Cython

In [1]:
# constants.py

UNIVERSE = int(
    '11111111'
    '11111111'
    '11111111'
    '11111111'
    '11111111'
    '11111111'
    '11111111'
    '11111111',
    2,
)

EMPTY = int(
    '00000000'
    '00000000'
    '00000000'
    '00000000'
    '00000000'
    '00000000'
    '00000000'
    '00000000',
    2,
)

RANK_FOUR = int(
    '00000000'
    '00000000'
    '00000000'
    '00000000'
    '11111111'
    '00000000'
    '00000000'
    '00000000',
    2
)

RANK_FIVE = int(
    '00000000'
    '00000000'
    '00000000'
    '11111111'
    '00000000'
    '00000000'
    '00000000'
    '00000000',
    2
)

RANK_FIVE = int(
    '00000000'
    '00000000'
    '00000000'
    '11111111'
    '00000000'
    '00000000'
    '00000000'
    '00000000',
    2
)

FILE_A = int(
    '10000000'
    '10000000'
    '10000000'
    '10000000'
    '10000000'
    '10000000'
    '10000000'
    '10000000',
    2
)

FILE_B = int(
    '01000000'
    '01000000'
    '01000000'
    '01000000'
    '01000000'
    '01000000'
    '01000000'
    '01000000',
    2
)

FILE_G = int(
    '00000010'
    '00000010'
    '00000010'
    '00000010'
    '00000010'
    '00000010'
    '00000010'
    '00000010',
    2
)

FILE_H = int(
    '00000001'
    '00000001'
    '00000001'
    '00000001'
    '00000001'
    '00000001'
    '00000001'
    '00000001',
    2
)

In [2]:
def get_starting_state(
    board=[
        ['♜', '♞', '♝', '♛', '♚', '♝', '♞', '♜'],
        ['♟︎', '♟︎', '♟︎', '♟︎', '♟︎', '♟︎', '♟︎', '♟︎'],
        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
        [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
        ['♙', '♙', '♙', '♙', '♙', '♙', '♙', '♙'],
        ['♖', '♘', '♗', '♕', '♔', '♗', '♘', '♖'],
    ]
):
    black_rook = 0
    black_knight = 0
    black_bishop = 0
    black_queen = 0
    black_king = 0
    black_pawn = 0

    white_rook = 0
    white_knight = 0
    white_bishop = 0
    white_queen = 0
    white_king = 0
    white_pawn = 0

    cursor = 1 << 63
    for row in board:
        for entry in row:
            if entry == '♜':
                black_rook |= cursor
            elif entry == '♞':
                black_knight |= cursor
            elif entry == '♝':
                black_bishop |= cursor
            elif entry == '♛':
                black_queen |= cursor
            elif entry == '♚':
                black_king |= cursor
            elif entry == '♟︎':
                black_pawn |= cursor
            elif entry == '♖':
                white_rook |= cursor
            elif entry == '♘':
                white_knight |= cursor
            elif entry == '♗':
                white_bishop |= cursor
            elif entry == '♕':
                white_queen |= cursor
            elif entry == '♔':
                white_king |= cursor
            elif entry == '♙':
                white_pawn |= cursor
            elif entry == ' ':
                pass
            else:
                raise ValueError(f'Invalid entry {entry}')
            cursor = cursor >> 1
    return {
        'white_player': 1,
        'black_rook': black_rook, 
        'black_knight': black_knight,
        'black_bishop': black_bishop,
        'black_queen': black_queen,
        'black_king': black_king,
        'black_pawn': black_pawn,
        'white_rook': white_rook,
        'white_knight': white_knight,
        'white_bishop': white_bishop,
        'white_queen': white_queen,
        'white_king': white_king,
        'white_pawn': white_pawn,
    }

def white(state):
    return (
        state['white_rook'] 
        | state['white_knight'] 
        | state['white_bishop'] 
        | state['white_queen'] 
        | state['white_king']
        | state['white_pawn']
    )


def black(state):
    return (
        state['black_rook'] 
        | state['black_knight'] 
        | state['black_bishop'] 
        | state['black_queen'] 
        | state['black_king']
        | state['black_pawn']
    )

def occupied(state):
    return white(state) | black(state)

In [3]:
# utils.py

def print_bitboard(b):
    for i in range(8):
        print('{:064b}'.format(b)[8*i:8*(i+1)])
        
        
starting_state = get_starting_state()
print_bitboard(starting_state['black_rook'])

10000001
00000000
00000000
00000000
00000000
00000000
00000000
00000000


In [4]:
def pop_count(x):
    count = 0
    for i in range(64):
        count += x & 1
        x >>= 1
    return count




In [6]:
2& 1

0

In [7]:
bb_to_idx = {}
idx_to_bb = {}
cursor = 1 << 63
for i in range(8):
    for j in range(8):
        bb_to_idx[cursor] = (i, j)
        idx_to_bb[(i, j)] = cursor
        cursor = cursor >> 1
def indicies(b):
    idxs = []
    while b:
        ls1b = b & -b  # isolate LS1B
        idxs.append(bb_to_idx[ls1b])
        b &= b - 1  # reset LS1B
    return idxs

In [8]:
indicies(starting_state['white_rook'])

[(7, 7), (7, 0)]

In [78]:
from collections import namedtuple

Move = namedtuple('Move', ['src', 'dst'])

# Pawn

In [ ]:
def white_pawn_attackset(pawn):
    

# King

In [80]:
king_attackset = {}
for i in range(8):
    for j in range(8):
        king_attackset[idx_to_bb[i, j]] = 0
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                to = (i + di, j + dj)
                if 0 <= to[0] <= 7 and 0 <= to[1] <= 7 and to != (i, j):
                    king_attackset[idx_to_bb[i, j]] |= idx_to_bb[to]

In [88]:
def king_moves(moves, king, color, other_attackset):
    attackset = king_attackset[king] & ~color & ~other_attackset
    while attackset:
        ls1b = attackset & -attackset
        moves.append(Move(src=king, dst=ls1b))
        attackset &= attackset - 1

In [89]:
moves = []
king_moves(moves, starting_state['white_king'], 0, 0)
moves
for move in moves:
    print_bitboard(move.dst)
    print()

00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000100

00000000
00000000
00000000
00000000
00000000
00000000
00000000
00010000

00000000
00000000
00000000
00000000
00000000
00000000
00000100
00000000

00000000
00000000
00000000
00000000
00000000
00000000
00001000
00000000

00000000
00000000
00000000
00000000
00000000
00000000
00010000
00000000



# Horse

In [26]:
knight_to = {}
for i in range(8):
    for j in range(8):
        knight_to[idx_to_bb[i, j]] = []
        for di, dj in [
            (-2, -1), (2, 1), (2, -1), (-2, 1),
            (-1, -2), (1, 2), (1, -2), (-1, 2),
        ]:
            to = (i + di, j + dj)
            if 0 <= to[0] <= 7 and 0 <= to[1] <= 7:
                knight_to[idx_to_bb[i, j]].append(idx_to_bb[to])

In [10]:
def knight_moves(moves, state):
    if state['white_player']:
        b = state['white_knight']
    else:
        b = state['black_knight']
    moves = []
    while b:
        ls1b = b & -b
        for to in knight_to[ls1b]:
            if to & (UNIVERSE - white(state)):
                moves.append({
                    'from': ls1b,
                    'to': to
                })
        b &= b - 1
    return moves

knight_moves([], starting_state)

[{'from': 2, 'to': 262144},
 {'from': 2, 'to': 65536},
 {'from': 64, 'to': 8388608},
 {'from': 64, 'to': 2097152}]

# Rook

In [11]:
rook_blockermask = {}
rook_magic_shifts = {}
cursor = 1 << 63
for i in range(8):
    for j in range(8):
        
        rook_blockermask[cursor] = 0
        rook_magic_shifts[cursor] = 0
        
        for step in range(1, i):
            to = (i - step, j)
            rook_blockermask[cursor] |= idx_to_bb[to]
            rook_magic_shifts[cursor] += 1
        
        for step in range(1, 7 - i):
            to = (i + step, j)
            rook_blockermask[cursor] |= idx_to_bb[to]
            rook_magic_shifts[cursor] += 1
            
        for step in range(1, j):
            to = (i, j - step)
            rook_blockermask[cursor] |= idx_to_bb[to]
            rook_magic_shifts[cursor] += 1
            
        for step in range(1, 7 - j):
            to = (i, j + step)
            rook_blockermask[cursor] |= idx_to_bb[to]
            rook_magic_shifts[cursor] += 1
            
        cursor = cursor >> 1

In [12]:
# Reference: https://rhysre.net/fast-chess-move-generation-with-magic-bitboards.html
# Source: https://github.com/GunshipPenguin/shallow-blue/blob/c6d7e9615514a86533a9e0ffddfc96e058fc9cfd/src/attacks.h#L120
MAGICS_ROOK = [
    0xa8002c000108020, 0x6c00049b0002001, 0x100200010090040, 0x2480041000800801, 0x280028004000800,
    0x900410008040022, 0x280020001001080, 0x2880002041000080, 0xa000800080400034, 0x4808020004000,
    0x2290802004801000, 0x411000d00100020, 0x402800800040080, 0xb000401004208, 0x2409000100040200,
    0x1002100004082, 0x22878001e24000, 0x1090810021004010, 0x801030040200012, 0x500808008001000,
    0xa08018014000880, 0x8000808004000200, 0x201008080010200, 0x801020000441091, 0x800080204005,
    0x1040200040100048, 0x120200402082, 0xd14880480100080, 0x12040280080080, 0x100040080020080,
    0x9020010080800200, 0x813241200148449, 0x491604001800080, 0x100401000402001, 0x4820010021001040,
    0x400402202000812, 0x209009005000802, 0x810800601800400, 0x4301083214000150, 0x204026458e001401,
    0x40204000808000, 0x8001008040010020, 0x8410820820420010, 0x1003001000090020, 0x804040008008080,
    0x12000810020004, 0x1000100200040208, 0x430000a044020001, 0x280009023410300, 0xe0100040002240,
    0x200100401700, 0x2244100408008080, 0x8000400801980, 0x2000810040200, 0x8010100228810400,
    0x2000009044210200, 0x4080008040102101, 0x40002080411d01, 0x2005524060000901, 0x502001008400422,
    0x489a000810200402, 0x1004400080a13, 0x4000011008020084, 0x26002114058042
]
rook_to_magic = {}
idx = 0
cursor = 1
for i in reversed(range(8)):
    for j in reversed(range(8)):
        rook_to_magic[cursor] = MAGICS_ROOK[idx]
        idx += 1
        cursor = cursor << 1

In [13]:
def get_blockers(blocker_idx, b):
    blockers = 0
    bits = pop_count(b)
    for i in range(bits):
        bit_index = (b&-b).bit_length()-1
        if blocker_idx & (1 << i):
            blockers |= (1 << bit_index)
        b &= b - 1  
    return blockers

In [14]:
def get_rook_attackset(pos, blockers):
    i, j = pos
    attackset = 0
    blocker_pos = indicies(blockers)
    for step in range(1, i + 1):
        to = (i - step, j)
        if to in blocker_pos:
            break
        attackset |= idx_to_bb[to]

    for step in range(1, 8 - i):
        to = (i + step, j)
        if to in blocker_pos:
            break
        attackset |= idx_to_bb[to]

    for step in range(1, j + 1):
        to = (i, j - step)
        if to in blocker_pos:
            break
        attackset |= idx_to_bb[to]

    for step in range(1, 8 - j):
        to = (i, j + step)
        if to in blocker_pos:
            break
        attackset |= idx_to_bb[to]
        
    return attackset

In [15]:
def get_rook_key(b, blockers):
    return (
        ((blockers * rook_to_magic[b]) % 2**64)
        >> (64 - rook_magic_shifts[b])
    )

In [16]:
from collections import defaultdict


ROOK_TABLE = defaultdict(dict)
cursor = 1 << 63
for i in range(8):
    for j in range(8):
        pos = (i, j)
        for blocker_idx in range(0, 1 << rook_magic_shifts[cursor]):
            blockers = get_blockers(blocker_idx, rook_blockermask[cursor])
            key = get_rook_key(cursor, blockers)
            new_attackset = get_rook_attackset(pos, blockers)
            old_attackset = ROOK_TABLE[cursor].get(key, None)
            if old_attackset is not None:
                assert old_attackset == new_attackset, cursor
            ROOK_TABLE[cursor][key] = new_attackset
            
        cursor = cursor >> 1

In [17]:
def rook_moves(moves, rook, blockers):
    while rook:
        ls1b = rook & -rook
        key = get_rook_key(ls1b, blockers & rook_blockermask[ls1b])
        new_move = ROOK_TABLE[ls1b][key]
        moves.append(new_move)
        rook &= rook - 1
        
        
moves = []
rook_moves(moves, starting_state['black_rook'], starting_state['white_pawn'])
for move in moves:
    print_bitboard(move)
    print()

11111110
00000001
00000001
00000001
00000001
00000001
00000000
00000000

01111111
10000000
10000000
10000000
10000000
10000000
00000000
00000000



# Bishop

In [18]:
bishop_blockermask = {}
bishop_magic_shifts = {}
cursor = 1 << 63
for i in range(8):
    for j in range(8):
        
        bishop_blockermask[cursor] = 0
        bishop_magic_shifts[cursor] = 0
        
        for step in range(1, min(7 - i, 7 - j)):
            to = (i + step, j + step)
            bishop_blockermask[cursor] |= idx_to_bb[to]
            bishop_magic_shifts[cursor] += 1
        
        for step in range(1, min(i, 7 - j)):
            to = (i - step, j + step)
            bishop_blockermask[cursor] |= idx_to_bb[to]
            bishop_magic_shifts[cursor] += 1
            
        for step in range(1, min(i, j)):
            to = (i - step, j - step)
            bishop_blockermask[cursor] |= idx_to_bb[to]
            bishop_magic_shifts[cursor] += 1
            
        for step in range(1, min(7 - i, j)):
            to = (i + step, j - step)
            bishop_blockermask[cursor] |= idx_to_bb[to]
            bishop_magic_shifts[cursor] += 1
        
            
        cursor = cursor >> 1

In [19]:
# Reference: https://rhysre.net/fast-chess-move-generation-with-magic-bitboards.html
# Source: https://github.com/GunshipPenguin/shallow-blue/blob/c6d7e9615514a86533a9e0ffddfc96e058fc9cfd/src/attacks.h#L136
MAGICS_BISHOP = [
    0x89a1121896040240, 0x2004844802002010, 0x2068080051921000, 0x62880a0220200808, 0x4042004000000,
    0x100822020200011, 0xc00444222012000a, 0x28808801216001, 0x400492088408100, 0x201c401040c0084,
    0x840800910a0010, 0x82080240060, 0x2000840504006000, 0x30010c4108405004, 0x1008005410080802,
    0x8144042209100900, 0x208081020014400, 0x4800201208ca00, 0xf18140408012008, 0x1004002802102001,
    0x841000820080811, 0x40200200a42008, 0x800054042000, 0x88010400410c9000, 0x520040470104290,
    0x1004040051500081, 0x2002081833080021, 0x400c00c010142, 0x941408200c002000, 0x658810000806011,
    0x188071040440a00, 0x4800404002011c00, 0x104442040404200, 0x511080202091021, 0x4022401120400,
    0x80c0040400080120, 0x8040010040820802, 0x480810700020090, 0x102008e00040242, 0x809005202050100,
    0x8002024220104080, 0x431008804142000, 0x19001802081400, 0x200014208040080, 0x3308082008200100,
    0x41010500040c020, 0x4012020c04210308, 0x208220a202004080, 0x111040120082000, 0x6803040141280a00,
    0x2101004202410000, 0x8200000041108022, 0x21082088000, 0x2410204010040, 0x40100400809000,
    0x822088220820214, 0x40808090012004, 0x910224040218c9, 0x402814422015008, 0x90014004842410,
    0x1000042304105, 0x10008830412a00, 0x2520081090008908, 0x40102000a0a60140,
]
bishop_to_magic = {}
idx = 0
cursor = 1
for i in reversed(range(8)):
    for j in reversed(range(8)):
        bishop_to_magic[cursor] = MAGICS_BISHOP[idx]
        idx += 1
        cursor = cursor << 1

In [20]:
def get_bishop_attackset(pos, blockers):
    i, j = pos
    attackset = 0
    blocker_pos = indicies(blockers)   

    for step in range(1, min(8 - i, 8 - j)):
        to = (i + step, j + step)
        if to in blocker_pos:
            break
        attackset |= idx_to_bb[to]

    for step in range(1, min(i + 1, 8 - j)):
        to = (i - step, j + step)
        if to in blocker_pos:
            break
        attackset |= idx_to_bb[to]

    for step in range(1, min(i + 1, j + 1)):
        to = (i - step, j - step)
        if to in blocker_pos:
            break
        attackset |= idx_to_bb[to]

    for step in range(1, min(8 - i, j + 1)):
        to = (i + step, j - step)
        if to in blocker_pos:
            break
        attackset |= idx_to_bb[to]
        
    return attackset

In [21]:
def get_bishop_key(b, blockers):
    return (
        ((blockers * bishop_to_magic[b]) % 2**64)
        >> (64 - bishop_magic_shifts[b])
    )

In [22]:
from collections import defaultdict


BISHOP_TABLE = defaultdict(dict)
cursor = 1 << 63
for i in range(8):
    for j in range(8):
        pos = (i, j)
        for blocker_idx in range(0, 1 << bishop_magic_shifts[cursor]):
            blockers = get_blockers(blocker_idx, bishop_blockermask[cursor])
            key = get_bishop_key(cursor, blockers)
            new_attackset = get_bishop_attackset(pos, blockers)
            old_attackset = BISHOP_TABLE[cursor].get(key, None)
            if old_attackset is not None:
                assert old_attackset == new_attackset, cursor
            BISHOP_TABLE[cursor][key] = new_attackset
            
        cursor = cursor >> 1

In [23]:
def bishop_moves(moves, bishop, blockers):
    while bishop:
        ls1b = bishop & -bishop
        key = get_bishop_key(ls1b, blockers & bishop_blockermask[ls1b])
        new_move = BISHOP_TABLE[ls1b][key]
        moves.append(new_move)
        bishop &= bishop - 1
        
        
moves = []
bishop_moves(moves, starting_state['black_bishop'], starting_state['white_pawn'])
for move in moves:
    print_bitboard(move)
    print()

00000000
00001010
00010001
00100000
01000000
10000000
00000000
00000000

00000000
01010000
10001000
00000100
00000010
00000001
00000000
00000000

